# AIFarm Sprite Generator — SDXL-Turbo + Pixel Art Style

Generates isometric pixel art sprites for the Claude Buddy farming simulation.

**Requirements:** Colab Pro+ with A100 GPU (40GB VRAM)

**Model:** `stabilityai/sdxl-turbo` — open license, 4-step inference, no auth needed

## Sprite Categories
1. **Characters**: 8 hoodie colors × 4 directions × 3 frames = 96 sprites
2. **Animals**: 6 types × 4 directions × 2 frames = 48 sprites
3. **Crops**: 6 types × 4 growth stages = 24 sprites
4. **Buildings**: 7 structures = 7 sprites
5. **Tiles**: 8 terrain types = 8 sprites

In [ ]:
# Cell 1: Mount Google Drive + Check GPU
from google.colab import drive
drive.mount('/content/drive')

# Create persistent directories on Drive
import os
DRIVE_ROOT = '/content/drive/MyDrive/AIFarm'
MODEL_CACHE = os.path.join(DRIVE_ROOT, 'models')
SPRITE_OUTPUT = os.path.join(DRIVE_ROOT, 'sprites')
os.makedirs(MODEL_CACHE, exist_ok=True)
os.makedirs(SPRITE_OUTPUT, exist_ok=True)
print(f'Drive mounted! Model cache: {MODEL_CACHE}')
print(f'Sprite output: {SPRITE_OUTPUT}')

# Check GPU
!nvidia-smi
print('---')
import psutil
print(f'RAM: {psutil.virtual_memory().total / 1e9:.1f} GB')

In [ ]:
# Cell 2: Install FLUX.1 dependencies
!pip install -q diffusers transformers accelerate safetensors sentencepiece protobuf
!pip install -q peft  # for LoRA support
!pip install -q Pillow
print('Dependencies installed!')

In [ ]:
# Cell 3: Load SDXL-Turbo (open model, no auth needed)
import torch
from diffusers import StableDiffusionXLPipeline

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    cache_dir=MODEL_CACHE,
)
pipe = pipe.to("cuda")
print("SDXL-Turbo loaded on A100! Ready for 4-step inference.")

In [ ]:
# Cell 4: Style constants — "Harvest Moon meets Animal Crossing" pixel art

BASE_STYLE = (
    "isometric 45 degree top-down view, pixel art style, "
    "16-bit SNES aesthetic, vibrant saturated colors, "
    "clean sharp pixels, no anti-aliasing, chunky pixels, thick outlines, "
    "transparent background, game sprite asset, "
    "harvest moon style, cozy farming game aesthetic"
)

# Hoodie colors matching Character.HOODIE_COLORS in character.js
HOODIE_COLORS = [
    ('blue',    '#5B8DD9'),
    ('red',     '#E8734A'),
    ('green',   '#6AB04C'),
    ('purple',  '#9B59B6'),
    ('orange',  '#F39C12'),
    ('teal',    '#1ABC9C'),
    ('pink',    '#E84393'),
    ('yellow',  '#F1C40F'),
]

DIRECTIONS = ['front', 'left', 'right', 'back']

ANIMALS = ['chicken', 'cow', 'pig', 'sheep', 'cat', 'dog']

CROPS = [
    ('carrot',     'orange carrot'),
    ('sunflower',  'yellow sunflower'),
    ('watermelon', 'green watermelon'),
    ('tomato',     'red tomato'),
    ('corn',       'yellow corn stalk'),
    ('pumpkin',    'orange pumpkin'),
]

BUILDINGS = [
    ('well',     'stone water well with wooden roof'),
    ('barn',     'red wooden barn with hay'),
    ('windmill', 'dutch windmill with rotating blades'),
    ('market',   'market stall with colorful awning'),
    ('clock',    'clock tower with bell'),
    ('townhall', 'town hall with columns and flag'),
    ('statue',   'stone memorial statue'),
]

TILES = ['grass', 'dirt', 'soil', 'water', 'stone', 'sand', 'path', 'flowers']

print(f'Configured: {len(HOODIE_COLORS)} colors, {len(ANIMALS)} animals, {len(CROPS)} crops, {len(BUILDINGS)} buildings')

In [ ]:
# Cell 5: Generation helper functions (SDXL-Turbo)
from PIL import Image
import os

OUTPUT_DIR = SPRITE_OUTPUT
LOCAL_DIR = '/content/sprites'
os.makedirs(LOCAL_DIR, exist_ok=True)

SEED = 42

def generate_sprite(prompt, filename, size=512, steps=4):
    """Generate a single sprite using SDXL-Turbo."""
    full_prompt = f"{prompt}, {BASE_STYLE}"
    generator = torch.Generator('cuda').manual_seed(SEED)

    image = pipe(
        prompt=full_prompt,
        width=size,
        height=size,
        num_inference_steps=steps,
        generator=generator,
        guidance_scale=0.0,
    ).images[0]

    # Save to both local and Drive
    local_path = os.path.join(LOCAL_DIR, filename)
    drive_path = os.path.join(OUTPUT_DIR, filename)
    image.save(local_path)
    image.save(drive_path)
    print(f'  Saved: {drive_path}')
    return image

def generate_sprite_sheet(prompt_template, name, directions, frames_per_dir, frame_size=48):
    """Generate individual frames and assemble into a sprite sheet."""
    sheet_w = frames_per_dir * frame_size
    sheet_h = len(directions) * frame_size
    sheet = Image.new('RGBA', (sheet_w, sheet_h), (0, 0, 0, 0))

    for d_idx, direction in enumerate(directions):
        for f_idx in range(frames_per_dir):
            action = 'standing' if f_idx == 0 else f'walking frame {f_idx}'
            prompt = prompt_template.format(direction=direction, action=action)

            img = generate_sprite(prompt, f'{name}_{direction}_{f_idx}.png')
            if img:
                img_resized = img.resize((frame_size, frame_size), Image.NEAREST)
                sheet.paste(img_resized, (f_idx * frame_size, d_idx * frame_size))

    for out_dir in [LOCAL_DIR, OUTPUT_DIR]:
        sheet_path = os.path.join(out_dir, f'{name}_sheet.png')
        sheet.save(sheet_path)
    print(f'Sheet saved: {OUTPUT_DIR}/{name}_sheet.png ({sheet_w}x{sheet_h})')
    return sheet

print('Helper functions ready! SDXL-Turbo, 4-step inference.')

In [ ]:
# Cell 6: TEST — Generate a single character sprite to verify quality
test_prompt = (
    "a tiny chibi farmer character wearing a blue hoodie, "
    "facing front, standing pose, 48x48 pixel resolution, "
    f"{BASE_STYLE}"
)

generator = torch.Generator('cuda').manual_seed(42)
test_img = pipe(
    prompt=test_prompt,
    width=512,
    height=512,
    num_inference_steps=4,
    generator=generator,
    guidance_scale=0.0,
).images[0]

test_img.save('/content/sprites/test_character.png')
display(test_img)
print('Test sprite generated! Check quality before proceeding.')

In [ ]:
# Cell 7: Generate all CHARACTER sprite sheets (8 colors × 4 dirs × 3 frames)
# WARNING: This takes ~30-60 minutes on A100

for color_name, color_hex in HOODIE_COLORS:
    print(f'\n=== Generating character: {color_name} hoodie ===')
    template = (
        f"a tiny chibi farmer character wearing a {color_name} hoodie, "
        "facing {direction}, {action}, "
        "48x48 pixel resolution"
    )
    generate_sprite_sheet(
        template,
        f'char_{color_name}',
        directions=DIRECTIONS,
        frames_per_dir=3,
        frame_size=48,
    )

print('\nAll character sheets generated!')

In [ ]:
# Cell 8: Generate ANIMAL sprite sheets (6 types × 4 dirs × 2 frames)

ANIMAL_PROMPTS = {
    'chicken': 'a tiny white chicken',
    'cow':     'a tiny white and black spotted cow',
    'pig':     'a tiny pink pig',
    'sheep':   'a tiny fluffy white sheep',
    'cat':     'a tiny orange tabby cat',
    'dog':     'a tiny brown shiba inu dog',
}

for animal in ANIMALS:
    print(f'\n=== Generating animal: {animal} ===')
    desc = ANIMAL_PROMPTS[animal]
    template = (
        f"{desc}, farm animal, "
        "facing {direction}, {action}, "
        "32x32 pixel resolution"
    )
    generate_sprite_sheet(
        template,
        f'animal_{animal}',
        directions=DIRECTIONS,
        frames_per_dir=2,
        frame_size=32,
    )

print('\nAll animal sheets generated!')

In [ ]:
# Cell 9: Generate CROP sprites (6 types × 4 stages)

STAGE_NAMES = ['seed in soil', 'small green sprout', 'growing plant with leaves', 'fully mature ready to harvest']

for crop_id, crop_desc in CROPS:
    print(f'\n=== Generating crop: {crop_id} ===')
    for stage_idx, stage_name in enumerate(STAGE_NAMES):
        prompt = (
            f"a {crop_desc} farm crop, {stage_name}, "
            "planted in tilled soil, "
            "32x32 pixel resolution"
        )
        generate_sprite(prompt, f'crop_{crop_id}_stage{stage_idx}.png', size=512, steps=4)

print('\nAll crop sprites generated!')

In [ ]:
# Cell 10: Generate BUILDING sprites (7 buildings)

for bld_id, bld_desc in BUILDINGS:
    print(f'\n=== Generating building: {bld_id} ===')
    prompt = (
        f"a {bld_desc}, small cozy farm building, "
        "64x64 pixel resolution"
    )
    generate_sprite(prompt, f'building_{bld_id}.png', size=512, steps=4)

print('\nAll building sprites generated!')

In [ ]:
# Cell 11: Generate TILE sprites (8 terrain types)

TILE_PROMPTS = {
    'grass':   'lush green grass tile, diamond isometric shape',
    'dirt':    'brown dirt tile, diamond isometric shape',
    'soil':    'dark tilled farm soil tile, diamond isometric shape',
    'water':   'blue pond water tile, diamond isometric shape with reflections',
    'stone':   'grey cobblestone tile, diamond isometric shape',
    'sand':    'golden beach sand tile, diamond isometric shape',
    'path':    'beige walking path tile, diamond isometric shape',
    'flowers': 'green grass with wildflowers tile, diamond isometric shape',
}

for tile_id, tile_desc in TILE_PROMPTS.items():
    print(f'\n=== Generating tile: {tile_id} ===')
    prompt = (
        f"{tile_desc}, terrain ground tile, "
        "32x16 pixel resolution, 2:1 isometric ratio"
    )
    generate_sprite(prompt, f'tile_{tile_id}.png', size=512, steps=4)

print('\nAll tile sprites generated!')

In [ ]:
# Cell 12: Package all sprites into a zip for download
import shutil

shutil.make_archive('/content/aifarm-sprites', 'zip', OUTPUT_DIR)
print('Sprites packaged: /content/aifarm-sprites.zip')
print(f'Total files: {len(os.listdir(OUTPUT_DIR))}')

# Download link (Colab-specific)
from google.colab import files
files.download('/content/aifarm-sprites.zip')

## Alternative: google.colab.ai (Imagen 3)
If FLUX.1 is too slow or quality isn't right, try Google's built-in image generation.
This doesn't need GPU — it's API-based.

In [ ]:
# Cell 13: Alternative — test google.colab.ai for image generation
# Uncomment below to test Imagen 3 (no GPU needed)

# from google.colab import ai
# response = ai.generate_image(
#     "isometric 45 degree pixel art, tiny chibi farmer wearing blue hoodie, "
#     "harvest moon game style, 48x48 sprite, transparent background"
# )
# display(response)